<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/data_cleaning/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Skript um die Messdaten un Vorhersagedaten zusammen zu fühern und dieses zu bereinigen. Zudem werden weitere Features die für das Trainig des Netzwerks wichtig sind erzeugt.

Dateiname: Data_Cleaning

[Notebook 0: Data Cleaning](./0_DataCleaning.ipynb)

[Notebook 1: EDA](./1_EDA_and_Cleaning.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling_and_Predictions.ipynb)

[Notebook 3: Technical Report](./3_Technical_Report.ipynb)

## Installation der Bibliothek PVlib:
Dieses Modul wird für die Berechnung des Sonnenstandwinkels verwendet.

In [1]:
# Installation der pvlib um den Sonnenstandswinkel zu berechnen
!pip install pvlib

## Import der benötigten Module und allgemeines Setup:

In [2]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# import der benötigten Module

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pvlib

from datetime import datetime
from datetime import timedelta

import matplotlib
import seaborn as sns

In [4]:
# Standort der Messstaion für die Berechnung des Sonnenstandswinkel
latitude = 50.8
longitude = 7.2

# Angaben um das Datum und die Uhrzeit in Sin und Cos zu codieren
seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

In [5]:
# Pfad zu den Messwerten / Vorhersagewerten und dem Speicherort auf Google Drive
# Import
folder_UVI = '/content/drive/My Drive/Colab_Notebooks/CSV_UVI/'
folder_Solys = '/content/drive/My Drive/Colab_Notebooks/SOLYS_CSV/'
folder_CAMS = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'
folder_VarIdx = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'

name_UVI = ['22.06', '22.07', '22.08', '22.09', '22.10', '22.11', '22.12', '23.01', '23.02', '23.03', '23.04', '23.05', '23.07', '23.08']  # Hier wird angegeben, welche Monate importiert werden sollen
name_Solys = 'Solys_CSV'
name_CAMS = 'CAMS_std_CSV'
name_CAMS_Glo = 'CAMS_Glo_CSV'
name_CAMS_TCC = 'CAMS_TCC_CSV'
name_VarIdx = 'blabla'

# Export

## Import der UVI-Messdaten:

Die Messdaten sind in CSV-Dateien gespeichert und müssen importiert werden.

In [6]:
## Code zum Import der Messdaten
dataframes = []
df_UVI_combined = []

for name in name_UVI:
  file_path = folder_UVI + name
  # print(file_path)
  df_import = pd.read_csv(file_path)
  dataframes.append(df_import)

df_UVI_combined = pd.concat(dataframes, ignore_index=True)
df_UVI_combined['Datetime'] = pd.to_datetime(df_UVI_combined['Datetime'])

In [7]:
# df_UVI_combined.fillna(0)

## Import der Solys-Messdaten:

Messdaten mit einer 2-minütigen Auflösung. Die Daten sind noch nicht reduziert.

In [8]:
# Einlesen der Solys-Messdaten:
df_Solys = pd.read_csv(folder_Solys + name_Solys)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_Solys['Datetime'] = pd.to_datetime(df_Solys['Datetime'])
df_Solys.set_index('Datetime', inplace = True)

In [9]:
# df_Solys

## Import der CAMS-Vorhersagedaten:

Die Daten sind noch nicht reduziert und es gibt für jede Stunde des Tages eine Vorhersage.

In [10]:
# Einlesen der Solys-Messdaten:
df_CAMS = pd.read_csv(folder_CAMS + name_CAMS)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_CAMS['Datetime'] = pd.to_datetime(df_CAMS['Datetime'])
df_CAMS.set_index('Datetime', inplace = True)

In [11]:
# df_CAMS

## Import der CAMS-Globalstrahlungs-Vorhersagedaten:

In [12]:
# Einlesen der Solys-Messdaten:
df_CAMS_GLO = pd.read_csv(folder_CAMS + name_CAMS_Glo)

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
df_CAMS_GLO['Datetime'] = pd.to_datetime(df_CAMS_GLO['Observation_period'])
df_CAMS_GLO.set_index('Datetime', inplace = True)
df_CAMS_GLO.drop(['Unnamed: 0', 'Observation_period'], axis = 1)

,Clear_sky_GHI,Clear_sky_BHI,GHI,BHI
Datetime,,,,
2022-06-01 00:01:00,0.0,0.0,0.0,0.0
2022-06-01 00:03:00,0.0,0.0,0.0,0.0
2022-06-01 00:05:00,0.0,0.0,0.0,0.0
2022-06-01 00:07:00,0.0,0.0,0.0,0.0
2022-06-01 00:09:00,0.0,0.0,0.0,0.0
...,...,...,...,...
2023-08-31 23:51:00,0.0,0.0,0.0,0.0
2023-08-31 23:53:00,0.0,0.0,0.0,0.0
2023-08-31 23:55:00,0.0,0.0,0.0,0.0


## Import der CAMS-TCC-Vorhersagedaten:

In [20]:
# Einlesen der Solys-Messdaten:
df_CAMS_TCC = pd.read_csv(folder_CAMS + name_CAMS_TCC, index_col=[0, 1])

# ! Der Index muss noch in Datetime umgewndelt werden !

# Die Spalte mit dem Zeitstempel in das Datetimeformat umwandeln und als Index speichern
# df_CAMS_TCC['Datetime'] = pd.to_datetime(df_CAMS_TCC['Datetime'])
# df_CAMS_TCC.set_index('Datetime', inplace = True)

KeyError: 'Datetime'

In [21]:
# df_CAMS_TCC

tcc
Datetime            coords                
2022-05-01 00:00:00 (2.0, 55.0)   1.000008
                    (2.0, 54.6)   0.983405
                    (2.0, 54.2)   0.981605
                    (2.0, 53.8)   0.853729
                    (2.0, 53.4)   0.663335
...                                    ...
2023-05-31 23:00:00 (12.0, 46.6)  0.048037
                    (12.0, 46.2)  0.152428
                    (12.0, 45.8)  0.246214
                    (12.0, 45.4)  0.216122
                    (12.0, 45.0)  0.184107

[6424704 rows x 1 columns]

## Bereinigen der Messdaten

Hier wird erklärt, was zum Bereinigen der Messdaten getan werden muss.

- Fehlende Messtage müssen ersetzt werden:
  - Prüfen, ob die Messwerte zusammenhängen.
  - Hinzufügen neuer Zeilen, falls die Messwerte nicht zusammen Hängen.

**Hier wird geprüft, ob die Messwerte zusammenhängend sind:**

In [ ]:
def insert_missing_rows(df):
    # Sortieren des DataFrame nach 'Datetime'
    df.sort_values(by='Datetime', inplace=True)

    # Initialisieren einer Liste, um die Zeilen mit fehlenden Daten einzufügen
    rows_to_insert = []

    # Gruppieren des DataFrame nach 'Datum'
    grouped = df.groupby('Datum')

    for date, group in grouped:
        # Sortieren der Gruppe nach 'Datetime'
        group.sort_values(by='Datetime', inplace=True)

        for i in range(1, len(group)):
            current_time = group.iloc[i]['Datetime']
            prev_time = group.iloc[i - 1]['Datetime']
            time_diff = current_time - prev_time

            if time_diff > timedelta(minutes=2):
                while prev_time + timedelta(minutes=2) < current_time:
                    prev_time += timedelta(minutes=2)
                    new_row = {
                        'Datetime': prev_time,
                        'Datum': date,
                        'Uhrzeit': prev_time.time(),
                        'Messzeitpunkt': (prev_time - prev_time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds(),
                        'erythem': 0,
                        'UVI': 0,
                        'DiffGreater2': 1,
                    }
                    rows_to_insert.append(new_row)

    # Einfügen der fehlenden Zeilen in einen DataFrame
    if rows_to_insert:
        df = df.append(rows_to_insert, ignore_index=True)

    # Sortieren des DataFrame nach 'Datetime'
    df.sort_values(by='Datetime', inplace=True)
    df = df.reset_index(drop = True)
    df['DiffGreater2'] = df['DiffGreater2'].fillna(0)

    return df

In [ ]:
df_UVI_WRows = pd.DataFrame()
df_UVI_WRows = insert_missing_rows(df_UVI_combined)

## Sonnenstandswinkel hinzufügen

In [ ]:
def calculate_solar_zenith_angle(dataframe, date_column, latitude, longitude, altitude=0):

    # Kopiere das ursprüngliche DataFrame, um es nicht zu ändern.
    result_df = dataframe.copy()

    # Konvertiere die Datumsspalte in einen datetime-Datentyp, falls sie es nicht bereits ist.
    if not pd.api.types.is_datetime64_any_dtype(dataframe[date_column]):
        result_df[date_column] = pd.to_datetime(dataframe[date_column])

    # Iteriere über die Zeilen des DataFrames und berechne den Solarzenitwinkel für jedes Datum.
    solar_zenith_angles = []
    for date in result_df[date_column]:
        solar_position = pvlib.solarposition.get_solarposition(date, latitude, longitude, altitude)
        solar_zenith_angle = solar_position['zenith'].values[0]
        solar_zenith_angles.append(solar_zenith_angle)

    # Füge die berechneten Solarzenitwinkel dem DataFrame hinzu.
    result_df['SZA'] = solar_zenith_angles

    return result_df

In [ ]:
df_UVI_WRows_SZ = pd.DataFrame()
df_UVI_WRows_SZ = calculate_solar_zenith_angle(df_UVI_WRows, 'Datetime', latitude, longitude)

## Zeit und Datum in Sin und Cos codieren
- Cyclical Encoding der Messzeit als Sinus und Cosinus

In [ ]:
def calculate_date_in_sine_cosine(dataframe, day, year):

    result_df = dataframe.copy()

    result_df['time_sin'] = np.sin(2*np.pi*result_df['Messzeitpunkt']/day)
    result_df['time_cos'] = np.cos(2*np.pi*result_df['Messzeitpunkt']/day)
    result_df['date_sin'] = np.sin((2*np.pi*result_df['Datetime'].dt.dayofyear * 24 * 60 * 60) / year)
    result_df['date_cos'] = np.cos((2*np.pi*result_df['Datetime'].dt.dayofyear * 24 * 60 * 60) / year)

    return result_df

In [ ]:
df_UVI_SZ_SC = calculate_date_in_sine_cosine(df_UVI_WRows_SZ, seconds_in_day, seconds_in_year)

## Erstellen einer Liste mit Stunden an denen es Messwerte gibt:

In [ ]:
# Herausschreiben des Datums und der Stunde aus den Messdaten um im Folgeden nur die relevanten Vorhersagedaten auszuwählen
df_UVI_SZ_SC['Date'] = df_UVI_SZ_SC['Datetime'].dt.date
df_UVI_SZ_SC['Hour'] = df_UVI_SZ_SC['Datetime'].dt.hour

In [ ]:
df_date_std = df_UVI_SZ_SC.groupby(['Date', 'Hour']).size().reset_index(name='Count')

In [ ]:
df_date_std['Datetime'] = pd.to_datetime(df_date_std['Date'].astype(str) + ' ' + df_date_std['Hour'].astype(str) + ':00:00')

In [ ]:
df_date_std.set_index('Datetime', inplace=True)

## Erstellen einer Liste mit Zeitstemplen an denen es Messwerte gibt:

In [ ]:
df_date = pd.DataFrame(pd.to_datetime(df_UVI_SZ_SC['Datetime']))
df_date.set_index('Datetime', inplace=True)

## Import der Vorhersagedaten und löschen nicht gebrauchter Einträge:

In [ ]:
dateiname = 'CSV_Cams_std'
df_cams_std = pd.read_csv(pickle_path + dateiname)
df_cams_std['Datetime'] = pd.to_datetime(df_cams_std['Datetime'])

In [ ]:
# Datetime als Index festlegen
df_cams_std.set_index('Datetime', inplace=True)

In [ ]:
# Löschen aller nicht relevanten Einträge
df_cams_std = df_cams_std[df_cams_std.index.isin(df_date_std.index)]

## Löschen nicht gebrauchter Solys Strahlungsdaten
Die Strahlungsdaten werden an die UVI-Messdaten angepasst

In [ ]:
df_Solys_std = pd.merge_asof(df_date, df_Solys, left_index=True, right_index=True, tolerance=pd.Timedelta('1min'))

In [ ]:
df_Solys_std

## Abspeichern des DataFrames als CSV

In [ ]:
df_UVI_SZ_SC.isnull().sum()

In [ ]:
# Irradiance metrics over time
df_UVI_SZ_SC[['UVI']].plot()
plt.title('UVI über die Zeit');

In [ ]:
def export_dataframes_to_csv(df1, df2, df3, file1_name, file2_name, file3_name, folder_name):

    try:
        # Exportiere den ersten DataFrame in eine CSV-Datei
        df1.to_csv(folder_name + file1_name)
        print(f'DataFrame 1 wurde erfolgreich in "{file1_name}" exportiert.')

        # Exportiere den zweiten DataFrame in eine CSV-Datei
        df2.to_csv(folder_name + file2_name)
        print(f'DataFrame 2 wurde erfolgreich in "{file2_name}" exportiert.')

        # Exportiere den dritten DataFrame in eine CSV-Datei
        df3.to_csv(folder_name + file3_name)
        print(f'DataFrame 3 wurde erfolgreich in "{file3_name}" exportiert.')

    except Exception as e:
        print(f'Fehler beim Export der DataFrames: {str(e)}')

In [ ]:
export_dataframes_to_csv(df_UVI_SZ_SC, df_Solys_std, df_cams_std, 'UVI_Messdaten.csv', 'Solys_Messdaten.csv', 'Vorhersage.csv', save_folder)